In [0]:
from pyspark.sql import SparkSession,Row
spark = SparkSession.builder.appName('Spark').getOrCreate()

jsonString="""{"Zipcode":704,"ZipCodeType":"STANDARD","City":"PARC PARQUE","State":"PR"}"""
df=spark.createDataFrame([(1, jsonString)],["id","value"])
df.show(truncate=False)

+---+--------------------------------------------------------------------------+
id |value |
+---+--------------------------------------------------------------------------+
1 |{"Zipcode":704,"ZipCodeType":"STANDARD","City":"PARC PARQUE","State":"PR"}|
+---+--------------------------------------------------------------------------+

In [0]:
#Convert JSON string column to Map type
from pyspark.sql.types import MapType,StringType
from pyspark.sql.functions import from_json
df2=df.withColumn("value",from_json(df.value,MapType(StringType(),StringType())))
df2.printSchema()
df2.show(truncate=False)

root
-- id: long (nullable = true)
-- value: map (nullable = true)
 |-- key: string
 |-- value: string (valueContainsNull = true)

+---+---------------------------------------------------------------------------+
id |value |
+---+---------------------------------------------------------------------------+
1 |{Zipcode -> 704, ZipCodeType -> STANDARD, City -> PARC PARQUE, State -> PR}|
+---+---------------------------------------------------------------------------+

In [0]:
from pyspark.sql.functions import to_json,col
df2.withColumn("value",to_json(col("value"))) \
   .show(truncate=False)

+---+----------------------------------------------------------------------------+
id |value |
+---+----------------------------------------------------------------------------+
1 |{"Zipcode":"704","ZipCodeType":"STANDARD","City":"PARC PARQUE","State":"PR"}|
+---+----------------------------------------------------------------------------+

In [0]:
from pyspark.sql.functions import json_tuple
df.select(col("id"),json_tuple(col("value"),"Zipcode","ZipCodeType","City")) \
    .toDF("id","Zipcode","ZipCodeType","City") \
    .show(truncate=False)

+---+-------+-----------+-----------+
id |Zipcode|ZipCodeType|City |
+---+-------+-----------+-----------+
1 |704 |STANDARD |PARC PARQUE|
+---+-------+-----------+-----------+

In [0]:
from pyspark.sql.functions import get_json_object
df.select(col("id"),get_json_object(col("value"),"$.ZipCodeType").alias("ZipCodeType")) \
    .show(truncate=False)

+---+-----------+
id |ZipCodeType|
+---+-----------+
1 |STANDARD |
+---+-----------+

In [0]:
from pyspark.sql.functions import schema_of_json,lit
schemaStr=spark.range(1) \
    .select(schema_of_json(lit("""{"Zipcode":704,"ZipCodeType":"STANDARD","City":"PARC PARQUE","State":"PR"}"""))) \
    .collect()[0][0]
print(schemaStr)

STRUCT<`City`: STRING, `State`: STRING, `ZipCodeType`: STRING, `Zipcode`: BIGINT>

In [0]:
from pyspark.sql import SparkSession,Row
spark = SparkSession.builder.appName('Spark').getOrCreate()

#read json from text file
dfFromTxt=spark.read.text("/FileStore/tables/zipcodes_json-1.txt")
dfFromTxt.printSchema()

root
-- value: string (nullable = true)

In [0]:
# Create Schema of the JSON column
from pyspark.sql.types import StructType,StructField, StringType
schema = StructType([ 
    StructField("Zipcode",StringType(),True), 
    StructField("ZipCodeType",StringType(),True), 
    StructField("City",StringType(),True), 
    StructField("State", StringType(), True)
  ])

In [0]:
#Convert json column to multiple columns
from pyspark.sql.functions import col,from_json
dfJSON = dfFromTxt.withColumn("jsonData",from_json(col("value"),schema)) \
                   .select("jsonData.*")
dfJSON.printSchema()
dfJSON.show(truncate=False)

root
-- Zipcode: string (nullable = true)
-- ZipCodeType: string (nullable = true)
-- City: string (nullable = true)
-- State: string (nullable = true)

+-------+-----------+-------------------+-----+
Zipcode|ZipCodeType|City |State|
+-------+-----------+-------------------+-----+
704 |STANDARD |PARC PARQUE |PR |
704 |STANDARD |PASEO COSTA DEL SUR|PR |
709 |STANDARD |BDA SAN LUIS |PR |
76166 |UNIQUE |CINGULAR WIRELESS |TX |
76177 |STANDARD |FORT WORTH |TX |
76177 |STANDARD |FT WORTH |TX |
704 |STANDARD |URB EUGENE RICE |PR |
85209 |STANDARD |MESA |AZ |
85210 |STANDARD |MESA |AZ |
32046 |STANDARD |HILLIARD |FL |
+-------+-----------+-------------------+-----+

In [0]:
# Alternatively using select
dfFromTxt.select(from_json(col("value"), schema).alias("data")) \
         .select("data.*") \
         .show()

+-------+-----------+-------------------+-----+
Zipcode|ZipCodeType| City|State|
+-------+-----------+-------------------+-----+
 704| STANDARD| PARC PARQUE| PR|
 704| STANDARD|PASEO COSTA DEL SUR| PR|
 709| STANDARD| BDA SAN LUIS| PR|
 76166| UNIQUE| CINGULAR WIRELESS| TX|
 76177| STANDARD| FORT WORTH| TX|
 76177| STANDARD| FT WORTH| TX|
 704| STANDARD| URB EUGENE RICE| PR|
 85209| STANDARD| MESA| AZ|
 85210| STANDARD| MESA| AZ|
 32046| STANDARD| HILLIARD| FL|
+-------+-----------+-------------------+-----+

In [0]:
#read json from csv file
dfFromCSV=spark.read.option("header",True) \
               .csv("/FileStore/tables/zipcodes-3.csv")
dfFromCSV.printSchema()
dfFromCSV.show(truncate=False)

root
-- Id: string (nullable = true)
-- Zipcode: string (nullable = true)
-- ZipCodeType: string (nullable = true)
-- City: string (nullable = true)
-- State: string (nullable = true)

+---+-------+-----------+-------------------+-----+
Id |Zipcode|ZipCodeType|City |State|
+---+-------+-----------+-------------------+-----+
1 |704 |STANDARD |PARC PARQUE |PR |
2 |704 |STANDARD |PASEO COSTA DEL SUR|PR |
3 |709 |STANDARD |BDA SAN LUIS |PR |
4 |76166 |UNIQUE |CINGULAR WIRELESS |TX |
5 |76177 |STANDARD |FORT WORTH |TX |
6 |76177 |STANDARD |FT WORTH |TX |
7 |704 |STANDARD |URB EUGENE RICE |PR |
8 |85209 |STANDARD |MESA |AZ |
9 |85210 |STANDARD |MESA |AZ |
10 |32046 |STANDARD |HILLIARD |FL |
+---+-------+-----------+-------------------+-----+

In [0]:
#Read json from string
data= [(""" {"Zipcode":704,"ZipCodeType":"STANDARD","City":"PARC PARQUE","State":"PR"} """)]
rdd = spark.sparkContext.parallelize(data)
df2 = spark.read.json(rdd)
df2.show()

+-----------+-----+-----------+-------+
 City|State|ZipCodeType|Zipcode|
+-----------+-----+-----------+-------+
PARC PARQUE| PR| STANDARD| 704|
+-----------+-----+-----------+-------+